In [9]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [10]:
#reading rheast and extracting districts
rheast= pd.read_csv('../zambia/data/rheas_model/maize_zmb_n_25_tamsat_cul_assimila_on_fertilizer5.csv')

pre=pd.read_csv('../zambia/data/preharvest/zambia_preharvest_data.csv')
rheast.rename(columns={'cname': 'district'},inplace=True)

#Add a year column to preharvest data and calculate yield
pre['year']=pre['season'].str.extract(r'(\d{4})')
pre['yield']=pre['production(mt)']/pre['area_planted']



#Extracting Districts
pre_list_districts= pre['district'].to_list()
pre_set_districts=set(pre_list_districts)
pre_districts=list(pre_set_districts)
rheast_list_districts=rheast['district'].to_list()
rheast_set_districts=set(rheast_list_districts)
rheas_districts=list(rheast_set_districts)

In [11]:
#Finding common districts in rheas and preharvest rheast
districts=[]
for i in pre_districts:
    if i in rheas_districts:
        districts.append(i)

missing_districts=[]
for j in pre_districts:
    if j not in  rheas_districts:
        missing_districts.append(j)
# missing_districts
#
# print(rheast[rheast['district'].str.contains('Mi')])
# mask = np.column_stack(rheast['district'].str.contains(r"^\Ndola", na=False) for col in df])
# df.loc[mask.any(axis=1)]
# len(pre_districts)
# if rheast['district'].str.contains('Kapiri').any():
districts

['Kalulushi',
 'Luwingu',
 'Kitwe',
 'Lundazi',
 'Lusaka',
 'Zambezi',
 'Mansa',
 'Mporokoso',
 'Samfya',
 'Lukulu',
 'Kalomo',
 'Mufumbwe',
 'Mambwe',
 'Mpongwe',
 'Masaiti',
 'Kaputa',
 'Sinazongwe',
 'Kabompo',
 'Lufwanyama',
 'Livingstone',
 'Kazungula',
 'Katete',
 'Mazabuka',
 'Mafinga',
 'Ndola',
 'Luangwa',
 'Chadiza',
 'Kasama',
 'Mwense',
 'Sesheke',
 'Choma',
 'Solwezi',
 'Mwinilunga',
 'Mungwi',
 'Serenje',
 'Petauke',
 'Chongwe',
 'Nchelenge',
 'Chama',
 'Namwala',
 'Kafue',
 'Luanshya',
 'Mpika',
 'Chinsali',
 'Chingola',
 'Nakonde',
 'Kasempa',
 'Isoka',
 'Mongu',
 'Kaoma',
 'Chililabombwe',
 'Chibombo',
 'Nyimba',
 'Mufulira',
 'Kalabo',
 'Shangombo',
 'Kabwe',
 'Chilubi',
 'Kapiri Mposhi',
 'Kawambwa',
 'Senanga',
 'Siavonga',
 'Chavuma',
 'Mbala',
 'Mkushi',
 'Monze',
 'Chipata',
 'Gwembe',
 'Mumbwa',
 'Mpulungu',
 'Ikelenge']

In [12]:
#changing date columns
rheast['planting']=pd.to_datetime(rheast['planting'], format='ISO8601')
rheast['harvest'] = pd.to_datetime(rheast['harvest'], format='ISO8601')
rheast['year'] = rheast['planting'].dt.year
rheast['yield']=rheast['gwad']/1000
rheast


,district,planting,harvest,lai,wsgd,gwad,year,yield
0,Mpulungu,1998-01-01,1998-05-11,0.45,0.0,3753,1998,3.753
1,Ndola,1998-01-01,1998-05-11,0.29,0.0,3713,1998,3.713
2,Mulobezi,1998-01-01,1998-05-11,0.27,0.0,3607,1998,3.607
3,Nsama,1998-01-01,1998-05-11,0.40,0.0,3751,1998,3.751
4,Lunga,1998-01-01,1998-05-11,0.45,0.0,3753,1998,3.753
...,...,...,...,...,...,...,...,...
290374,Lusaka,2022-10-16,2022-12-30,0.68,0.0,65,2022,0.065
290375,Mansa,2022-10-16,2022-12-30,1.04,0.0,88,2022,0.088
290376,Mambwe,2022-10-16,2022-12-30,0.69,0.0,66,2022,0.066
290377,Mbala,2022-10-16,2022-12-30,0.99,0.0,84,2022,0.084


In [13]:
#Derive seasons from the data
seasons = [1, 1, 1, 1,1, 0, 0, 0, 0, 1, 1, 1]
month_to_season=dict(zip(range(1,13), seasons))
month_to_season
rheast['season']= rheast['harvest'].dt.month.map(month_to_season)

# Extract data only for the planting season
rheast_model= rheast.loc[rheast['season']==1]
rheast_data=rheast_model.groupby(['district','year', 'season'], as_index=False).agg({'yield': 'mean'})
#Filter rheast to merge preharvest i.e exclude 2006, 2007, 2011, 2013, and 2015
rheast_data1=rheast_data.loc[(rheast_data['year']>=2000) & (rheast_data['year']<=2016)]
rheast_data2=rheast_data1.drop(rheast_data1[(rheast_data1['year']==2006)|(rheast_data1['year']==2007)|(rheast_data1['year']==2011)
                       |(rheast_data1['year']==2013)|(rheast_data1['year']==2015)].index)
# rheast_data3=rheast_data2.reset_index(drop=True, inplace=True)

# (dataframe.drop(dataframe[dataframe['cost'] > 300].index))

pre1=pre.groupby(['district', 'year'], as_index=False).agg({'yield': 'mean'})
# pre1.to_csv('zambia_preharvest.csv')
# pre.loc[pre['district']=='Mpongwe']
# df=pre1.loc[pre1['district']=='Kalomo']
# df
# d=df.reset_index(drop=True, inplace=True)
# rheast_data[['district', 'year', 'yield']].to_csv('rheas_dssat_model.csv')


In [14]:

#LOOPING through the districts
# pre=pre.loc[pre['district']=='Chadiza'].reset_index(drop=True)
# rheast=rheast_data.loc[rheast_data['district']=='Chadiza'].reset_index(drop=True)
# #  pearson's correlation
# corr=pre['yield'].corr(rheast['yield'])


metrics=[]
for i in districts:
    pre1=pre1.loc[pre1['district']==i].reset_index(drop=True)
    rheast_data2=rheast_data2.loc[rheast_data2['district']==i].reset_index(drop=True)
    #  pearson's correlation
    corr=rheast_data2['yield'].corr(pre1['yield'])
    # # #root mean square error
    rmse = np.sqrt(((rheast_data2['yield']-pre1['yield'])**2).mean())
    # # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_data2['yield']-rheast_data2['yield'].mean())-(pre1['yield']-pre1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_data2['yield']-pre1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    metrics.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])
metrics
# pre=pre.loc[pre['district']=='Kalomo'].reset_index()
# #  correlation
# corr=pre['yield'].corr(rheast['yield'])
# pre
# dssat=dssat.loc[(dssat['year']==2011) | (dssat['year']==2012) | (dssat['year']==2014)]

# post['yield']=post['production']/post['hactares']
# dssat=dssat.loc[(dssat['year']==2011) | (dssat['year']==2012) | (dssat['year']==2014)]
# post
# post=post.reset_index()
# # dssat=dssat.reset_index()

# # correlation
# # dssat['yield'].corr(post['yield'])
# #rmse
# # pearson's correlation
# corr = dssat['yield'].corr(post['yield']) 
# # #root mean square error
# rmse = np.sqrt(((dssat['yield']-post['yield'])**2).mean())
# post['yield']
# # #unbiased RMSE
# ubrmse = np.sqrt((((dssat['yield']-dssat['yield'].mean())-(post['yield']-post['yield'].mean()))**2).mean())

# # #bias
# bias = (dssat['yield']-post['yield']).mean()

# # print(corr,rmse,ubrmse,bias)
statistics_data=pd.DataFrame(metrics)
stats=statistics_data.rename({0:'District',1: 'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# # stats.to_csv('preharvest_dssat_distric_level_metrics.csv')
# stats
# districts


,District,correlation,rmse,ubrmse,bias
0,Kalulushi,-0.533742,1.564068,0.751311,1.371802
1,Luwingu,NaN,NaN,NaN,NaN
2,Kitwe,NaN,NaN,NaN,NaN
3,Lundazi,NaN,NaN,NaN,NaN
4,Lusaka,NaN,NaN,NaN,NaN
...,...,...,...,...,...
66,Chipata,NaN,NaN,NaN,NaN
67,Gwembe,NaN,NaN,NaN,NaN
68,Mumbwa,NaN,NaN,NaN,NaN
69,Mpulungu,NaN,NaN,NaN,NaN


In [15]:
# Province Level
#Read shapefiles and rename columns
zambia_district_shp = gpd.read_file('data/shp/zambia_districts.shp')
zambia_shp = gpd.read_file('data/shp/gadm36_ZMB_0.shp')
zambia_province_shp = gpd.read_file('data/shp/gadm36_ZMB_1.shp')
zambia_district_shp .rename(columns={'DISTRICT':'district', 'PROVINCE':'province'}, inplace=True)
# merge the yields and shapefiles by provinces
pre_shp= pd.merge(zambia_district_shp, pre1, on="district")
rheast_shp= pd.merge(zambia_district_shp, rheast_data2, on="district")
# rheasc_shp= pd.merge(zambia_district_shp, rheast_data2 on="p")
pre_province=pre_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
rheast_province=rheast_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
rheast_province

#Extracting Provinces
pre_list_provinces= pre_province['province'].to_list()
pre_set_provinces=set(pre_list_provinces)
pre_provinces=list(pre_set_provinces)
rheast_list_provinces=rheast_province['province'].to_list()
rheast_set_provinces=set(rheast_list_provinces)
rheast_provinces=list(rheast_set_provinces)

#list of provinces
provinces=[]
for i in pre_provinces:
    if i in rheast_provinces:
        provinces.append(i)
# provinces


In [17]:
province_metrics=[]
for i in provinces:
    pre_province=pre_province.loc[pre_province['province']==i]#.reset_index(drop=True)
    rheast_province=rheast_province.loc[rheast_province['province']==i]#.reset_index(drop=True)  
    #  pearson's correlation
    corr=rheast_province['yield'].corr(pre_province['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province['yield']-pre_province['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province['yield']-rheast_data2['yield'].mean())-(pre_province['yield']-pre_province['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province['yield']-pre_province['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics=pd.DataFrame(province_metrics)
province_statistics=province_statistics.rename({0:'District',1: 'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics.to_csv('preharvest_dssat_province_level_metrics.csv')
province_statistics

""
